# Brasileirão Série A
### Previsão de resultados de partidas de futebol usando aprendizado de máquina

Um processo abrangente de modelagem e previsão de resultados de futebol no Brasil. 
Inicialmente, se extrai dados de uma API, incluindo informações de campeonatos e partidas, e ignora avisos de certificado SSL para garantir a conectividade. 
Em seguida, o código calcula o aproveitamento dos times, tanto geral quanto recente, e integra dados históricos de temporadas anteriores, bem como valores de mercado dos times a partir de um arquivo CSV. 
Para a modelagem, as partidas são enriquecidas para análise, calculando-se médias de gols marcados e sofridos, pontos acumulados e aproveitamento em confrontos diretos. 
Finalmente, o texto prepara os dados para modelos de Machine Learning, aplicando e avaliando classificadores como RandomForest e LogisticRegression utilizando validação Leave-One-Season-Out (LOSO) e Monte Carlo Cross-Validation (MCCV) para prever resultados futuros e exportar as projeções.

Obs.: Foi testado também GradientBoosting, KNN e NaiveBayes mas não tiveram bons resultados e foram removidos das validações LOSO e MCCV.

In [1]:
# Importando as bibliotecas
import requests
import urllib3
import pandas as pd
from IPython.display import display
import numpy as np
import time

start_time = time.time()

In [2]:
# Ignorar warnings de certificado SSL
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

url = "https://service.ig.com.br/football_ig/campeonatos/10/fases/768"
response = requests.get(url, verify=False)
data = response.json()

### Edição

In [3]:
# Dados da edição
edicao_data = {}
if "edicao" in data:
    for k, v in data["edicao"].items():
        edicao_data[f"edicao.{k}"] = v

### Tabela

In [4]:
# Tabela
tabela_data = []
if "tabela" in data:
    for posicao in data["tabela"]:
        registro = dict(posicao)
        registro.update(edicao_data)
        tabela_data.append(registro)

tabela_df = pd.json_normalize(tabela_data)

In [5]:
# Recalcula aproveitamento
tabela_df['aproveitamento'] = round((tabela_df['vitorias'] * 3 + tabela_df['empates']) / (tabela_df['jogos'] * 3), 4) *100

In [6]:
# Calcula aproveitamento recente
def calc_aproveitamento(ultimos):
    if not ultimos:
        return None
    pontos = sum(3 if r == "v" else 1 if r == "e" else 0 for r in ultimos)
    return round(pontos / 15, 4) * 100 if len(ultimos) >= 5 else None

if "ultimos_jogos" in tabela_df.columns:
    tabela_df["aproveitamento_recente"] = tabela_df["ultimos_jogos"].apply(calc_aproveitamento)

In [7]:
tabela_df = tabela_df.rename(columns={
        'edicao.edicao_id': 'edicao_id', 
        'edicao.temporada': 'edicao_temporada', 
        'edicao.nome':  'edicao_nome', 
        'edicao.nome_popular': 'edicao_nome_popular',
        'edicao.slug': 'edicao_slug', 
        'time.time_id': 'time_id', 
        'time.nome_popular': 'time_nome_popular', 
        'time.sigla': 'time_sigla', 
        'time.escudo': 'time_escudo'})

# print(tabela_df.columns)

In [8]:
colunas_ordenadas = ['edicao_id', 'edicao_temporada',
       'edicao_nome', 'edicao_nome_popular', 'edicao_slug', 'time_id',
       'time_nome_popular', 'time_sigla', 'time_escudo', 'posicao', 'pontos', 'jogos', 'vitorias', 'empates', 'derrotas',
       'gols_pro', 'gols_contra', 'saldo_gols', 'variacao_posicao', 'ultimos_jogos', 'aproveitamento', 'aproveitamento_recente',
       ]
colunas_exibicao = ['posicao', 'time_sigla', 'jogos', 'pontos', 'vitorias', 'empates', 'derrotas',
       'gols_pro', 'saldo_gols', 'aproveitamento', 'aproveitamento_recente',
       ]
colunas_exibicao_res = ['posicao', 'time_sigla', 'jogos', 'pontos', 'aproveitamento', 'aproveitamento_recente',
       ]

In [9]:
tabela_df = tabela_df[colunas_ordenadas]
tabela_df['data_atualizacao'] = pd.Timestamp.now()

In [10]:
# display(tabela_df[colunas_exibicao_res].sort_values(by="aproveitamento", ascending
# =False, inplace=False, ignore_index=True))

### Partidas

In [11]:
# Partidas
partidas_data = []
if "partidas" in data:
    for rodada, lista_partidas in data["partidas"].items():
        for partida in lista_partidas:
            registro = {"rodada": rodada}
            registro.update(partida)
            registro.update(edicao_data)
            partidas_data.append(registro)

partidas_df = pd.json_normalize(partidas_data)

In [12]:
# Remove colunas indesejadas em partidas_df
colunas_descartar_partidas = ["disputa_penalti", "slug", "hora_realizacao",
                              "data_realizacao_iso", "_link", "edicao.nome", "edicao.slug", "estadio",
                              ]
partidas_df = partidas_df.drop(columns=[c for c in colunas_descartar_partidas if c in partidas_df.columns])

In [13]:
# rodada int
def rodada_int(rodada):
    if isinstance(rodada, str):
        # Remove espaços, divide no 'a' e pega a primeira parte
        return rodada.strip().split('a')[0]
    return None  # Caso o valor não seja string

if "rodada" in partidas_df.columns:
    partidas_df["rodada_num"] = partidas_df["rodada"].apply(rodada_int).astype(int)

In [14]:
# add valor do time
valor_time_df = pd.read_csv("app/valor-time-titular.csv")
partidas_df['edicao.temporada'] = partidas_df['edicao.temporada'].astype('int64')
# mandante
partidas_df = partidas_df.merge(valor_time_df, left_on=['time_mandante.nome_popular', 'edicao.temporada'], right_on=['time', 'ano'], how='left')
partidas_df.drop(columns=['time', 'ano'], inplace=True)
partidas_df.rename(columns={'time.valor': 'time_mandante.valor'}, inplace=True)
# visitante
partidas_df = partidas_df.merge(valor_time_df, left_on=['time_visitante.nome_popular', 'edicao.temporada'], right_on=['time', 'ano'], how='left')
partidas_df.drop(columns=['time', 'ano'], inplace=True)
partidas_df.rename(columns={'time.valor': 'time_visitante.valor'}, inplace=True)

In [15]:
partidas_df["data_realizacao"] = pd.to_datetime(partidas_df["data_realizacao"], format="%d/%m/%Y")

In [16]:
# campeoanatos anteriores
historico_df = pd.read_csv("app/br-historico-2022-2024.csv")
historico_df['data_realizacao'] = pd.to_datetime(historico_df['data_realizacao'], unit='D', origin='1899-12-30')

In [17]:
# add registros antigos
partidas_df = pd.concat([partidas_df, historico_df], ignore_index=True)

In [18]:
colunas_mandante = ['rodada', 'rodada_num', 'data_realizacao', 'partida_id', 'estadio.estadio_id', 'estadio.nome_popular',
                    'status', 'edicao.edicao_id', 'edicao.temporada', 'edicao.nome_popular', 'campeonato.campeonato_id', 
                    'campeonato.nome', 'campeonato.slug', 'time_mandante.time_id', 'time_mandante.nome_popular',
                    'time_visitante.time_id', 'time_visitante.nome_popular', 'placar_mandante', 'placar_visitante',
                    'time_mandante.valor', 'time_visitante.valor',
                    ]
colunas_visitante = ['rodada', 'rodada_num', 'data_realizacao', 'partida_id', 'estadio.estadio_id', 'estadio.nome_popular',
                     'status', 'edicao.edicao_id', 'edicao.temporada', 'edicao.nome_popular', 'campeonato.campeonato_id', 
                     'campeonato.nome', 'campeonato.slug', 'time_mandante.time_id', 'time_mandante.nome_popular',
                     'time_visitante.time_id', 'time_visitante.nome_popular', 'placar_visitante', 'placar_mandante',
                     'time_mandante.valor', 'time_visitante.valor',
                     ]

In [19]:
# modelagem para as projeções
partidas_model_df = None
temp = None

for partida in partidas_df['partida_id']:
    # mandante
    temp = partidas_df[(partidas_df['partida_id'] == partida)]
    temp = temp[colunas_mandante]
    temp['mandante'] = 1
    temp.rename(columns={
        'time_mandante.time_id': 'time_id',
        'time_mandante.nome_popular': 'time',
        'time_visitante.time_id': 'adversario_id',
        'time_visitante.nome_popular': 'adversario',
        'time_mandante.valor': 'valor_time',
        'time_visitante.valor': 'valor_adversario',
        'placar_mandante': 'gols_time',
        'placar_visitante': 'gols_adversario',
    }, inplace=True)
    partidas_model_df = pd.concat([partidas_model_df, temp], ignore_index=True)

    # visitante
    temp = partidas_df[(partidas_df['partida_id'] == partida)]
    temp = temp[colunas_visitante]
    temp['mandante'] = 0
    temp.rename(columns={
        'time_mandante.time_id': 'adversario_id',
        'time_mandante.nome_popular': 'adversario',
        'time_visitante.time_id': 'time_id',
        'time_visitante.nome_popular': 'time',
        'time_mandante.valor': 'valor_adversario',
        'time_visitante.valor': 'valor_time',
        'placar_visitante': 'gols_time',
        'placar_mandante': 'gols_adversario',
    }, inplace=True)
    partidas_model_df = pd.concat([partidas_model_df, temp], ignore_index=True)

# classifica df
partidas_model_df.sort_values(by=['data_realizacao', 'rodada_num', 'partida_id', 'mandante'], ascending=[True, True, True, False], inplace=True)

partidas_agendadas_df = partidas_model_df[partidas_model_df['status'] != 'finalizado']
partidas_model_df = partidas_model_df[partidas_model_df['status'] == 'finalizado']       

In [20]:
partidas_model_df = pd.concat([partidas_model_df, partidas_agendadas_df[partidas_agendadas_df['rodada_num'] == 38]], ignore_index=True)

#### Enriquecimento dos dados
* Média de gols marcados time
* Média de gols sofridos time
* Média de gols marcados adversário
* Média de gols sofridos adversário
* Aproveitamento time
* Aproveitamento recente time
* Aproveitamento adversário
* Aproveitamento recente adversário
* Pontos time
* Pontos adversário
* Últimos confrontos

In [21]:
# Média de gols por time e temporada
# partidas_model_df[['edicao.temporada', 'time', 'gols_time', 'gols_adversario']].groupby(['edicao.temporada', 'time']).mean().reset_index()

In [22]:
partidas_model_df['ano-rodada'] = partidas_model_df['edicao.temporada'].astype(str) + "." + partidas_model_df['rodada_num'].astype(str)

# média de gols
media_gols_df = None
for rodada in partidas_model_df['ano-rodada'].drop_duplicates():
    temp = partidas_model_df[(partidas_model_df['rodada_num'] < int(rodada.split(".")[1])) &
                              (partidas_model_df['edicao.temporada'] == int(rodada.split(".")[0]))]
    temp = temp[['time', 'gols_time', 'gols_adversario']].groupby('time').mean().reset_index()
    temp['ano-rodada'] = rodada
    media_gols_df = pd.concat([media_gols_df, temp], ignore_index=True)

media_gols_df.rename(columns={'gols_time': 'media_gols_marcados', 'gols_adversario': 'media_gols_sofridos'}, inplace=True)

# add média time
partidas_model_df = partidas_model_df.merge(media_gols_df, on=['ano-rodada', 'time'], how='left')

# add média adversário
media_gols_df.rename(columns={'time': 'adversario', 'media_gols_marcados': 'media_gols_marcados_adversario', 'media_gols_sofridos': 'media_gols_sofridos_adversario'}, inplace=True)
partidas_model_df = partidas_model_df.merge(media_gols_df, on=['ano-rodada', 'adversario'], how='left')

In [23]:
# pontos conquistados
partidas_model_df['pontos'] = partidas_model_df.apply(
    lambda x: 3 if x['gols_time'] > x['gols_adversario'] else (1 if x['gols_time'] == x['gols_adversario'] else (0 if x['status'] != 'agendado' else np.nan)),
    axis=1
)

pontos_df = None
for rodada in partidas_model_df['ano-rodada'].drop_duplicates():
    temp = partidas_model_df[(partidas_model_df['rodada_num'] < int(rodada.split(".")[1])) &
                              (partidas_model_df['edicao.temporada'] == int(rodada.split(".")[0]))]
    temp = temp[['time', 'pontos']].groupby('time').sum().reset_index()
    temp['ano-rodada'] = rodada
    pontos_df = pd.concat([pontos_df, temp], ignore_index=True)

# add média time
pontos_df.rename(
    columns={
        'pontos': 'pontos_time'
    }, 
    inplace=True
)
partidas_model_df = partidas_model_df.merge(pontos_df, on=['ano-rodada', 'time'], how='left')

# add média adversario
pontos_df.rename(
    columns={
        'pontos_time': 'pontos_adversario',
        'time': 'adversario'
    }, 
    inplace=True
)
partidas_model_df = partidas_model_df.merge(pontos_df, on=['ano-rodada', 'adversario'], how='left')

In [24]:
# Aproveitamento

# cria a coluna "jogos" inicialmente como 0
partidas_model_df["jogos"] = np.nan

# conta só partidas já realizadas (por temporada e time)
partidas_model_df.loc[partidas_model_df["status"] != "agendado", "jogos"] = (
    partidas_model_df.loc[partidas_model_df["status"] != "agendado"]
    .groupby(["edicao.temporada", "time"])
    .cumcount()
)

# preenche os "agendados" com o valor anterior do mesmo time na mesma temporada
partidas_model_df["jogos"] = (
    partidas_model_df.groupby(["edicao.temporada", "time"])["jogos"]
    .ffill()
    .fillna(0)
    .astype(int)
)

# pontos acumulados até a rodada (se for ano-rodada já incluído, pode trocar)
partidas_model_df["pontos_acumulados"] = (
    partidas_model_df.groupby(["edicao.temporada", "time"])["pontos_time"]
    .cumsum()
)

# aproveitamento (%)
partidas_model_df["aproveitamento"] = (
    partidas_model_df["pontos_time"] / ((partidas_model_df["jogos"] +1 )* 3)
)


In [25]:
# Aproveitamento recente (últimas 5 partidas)
from collections import deque

window = 5

# ordena por temporada, time e rodada
partidas_model_df = partidas_model_df.sort_values(["edicao.temporada", "time", "rodada_num"])

# listas para armazenar resultados
pontos_recentes = []
jogos_validos = []

# processa temporada + time
for (temporada, time), g in partidas_model_df.groupby(["edicao.temporada", "time"]):
    last_points = deque()  # pontos das últimas 'window' partidas realizadas
    for idx, row in g.iterrows():
        # calcula soma e quantidade **antes de incluir a rodada atual**
        pontos_recentes.append(sum(last_points))
        jogos_validos.append(len(last_points))
        
        # adiciona os pontos da rodada atual somente se foi realizada
        if row["status"] != "agendado":
            last_points.append(row["pontos"])
        
        # mantém no máximo 'window' jogos realizados
        while len(last_points) > window:
            last_points.popleft()

# adiciona ao DataFrame
partidas_model_df["pontos_recentes"] = pontos_recentes
partidas_model_df["jogos_validos"] = jogos_validos
partidas_model_df["aproveitamento_recente"] = (
    partidas_model_df["pontos_recentes"] / (partidas_model_df["jogos_validos"] * 3)
)

# --- adiciona aproveitamento e aproveitamento recente para adversário ---
# seleciona as colunas do adversário, incluindo temporada
adversario_cols = ["edicao.temporada", "rodada_num", "time", "aproveitamento", "aproveitamento_recente"]

# renomeia as colunas do adversário
adversario_renomeado = partidas_model_df[adversario_cols].rename(
    columns={
        "time": "adversario",
        "aproveitamento": "aproveitamento_adversario",
        "aproveitamento_recente": "aproveitamento_recente_adversario"
    }
)

# merge com o próprio dataframe (mesma temporada, rodada e adversário)
partidas_model_df = partidas_model_df.merge(
    adversario_renomeado,
    on=["edicao.temporada", "rodada_num", "adversario"],
    how="left"
)


In [26]:
def calcular_confronto_recente(partidas_model_df, window=5, por_temporada=True):
    df = partidas_model_df.copy().reset_index(drop=True)
    # df = df[df['status']=='finalizado']

    # mantém apenas a visão do time
    A = df[["edicao.temporada", "rodada_num", "time", "adversario", "pontos", "status"]].copy()

    # ordena
    if por_temporada:
        sort_cols = ["edicao.temporada", "time", "adversario", "rodada_num"]
        group_cols = ["edicao.temporada", "time", "adversario"]
    else:
        sort_cols = ["time", "adversario", "edicao.temporada", "rodada_num"]
        group_cols = ["time", "adversario"]

    A = A.sort_values(sort_cols).reset_index(drop=True)

    # calcular aproveitamento nos últimos confrontos
    confronto_valores = {}
    for keys, g in A.groupby(group_cols, sort=False):
        last = deque()
        for idx, row in g.iterrows():
            if len(last) > 0:
                # aproveitamento = sum(last) / (len(last) * 3)
                aproveitamento = sum(last) / (len(last))
            else:
                aproveitamento = np.nan
            confronto_valores[idx] = aproveitamento

            if row["status"] != "agendado":
                last.append(int(row["pontos"]))
            if len(last) > window:
                last.popleft()

    A["confronto_recente"] = A.index.map(confronto_valores)

    # merge de volta ao df original
    out = df.merge(
        A[["edicao.temporada", "rodada_num", "time", "adversario", "confronto_recente"]],
        on=["edicao.temporada", "rodada_num", "time", "adversario"],
        how="left",
    )

    # adversario
    out_adv = out.copy()
    # out_adv.drop(labels='confronto_recente_adversario', inplace=True)
    out_adv = out_adv[
        [
            'edicao.temporada', 
            'rodada_num', 
            'time', 
            'adversario', 
            'confronto_recente', 
        ]
    ].rename(columns={'time': 'adversario',
            'adversario': 'time',
            'confronto_recente': 'confronto_recente_adversario'})
    
    # add confronto recente adversario
    out = out.merge(out_adv, on=['edicao.temporada', 'rodada_num', 'time', 'adversario'], how='left')

    out['confronto_recente'] = out['confronto_recente'].fillna(0)
    out['confronto_recente_adversario'] = out['confronto_recente_adversario'].fillna(0)
    
    return out


partidas_model_df = calcular_confronto_recente(partidas_model_df, window=5, por_temporada=False)
# por_temporada=True se quiser reiniciar o histórico a cada temporada

In [27]:
# pega a maior rodada por temporada + time
df_max_rodada = (
    partidas_model_df
    .groupby(['edicao.temporada', 'time'], as_index=False)['rodada_num']
    .max()
)

# reune com o dataframe original para trazer as demais colunas da linha da maior rodada
df_max_rodada = df_max_rodada.merge(
    partidas_model_df,
    on=['edicao.temporada', 'time', 'rodada_num'],
    how='left'
).reset_index(drop=True)

# display(df_max_rodada)

In [28]:
df_max_rodada = df_max_rodada[['edicao.temporada', 'time', 'media_gols_marcados', 'media_gols_sofridos', 'pontos', 
                               'pontos_time', 'jogos', 'pontos_acumulados', 'aproveitamento', 'pontos_recentes',
                               'jogos_validos', 'aproveitamento_recente', 'confronto_recente']]

In [29]:
# df_max_rodada[df_max_rodada['edicao.temporada'] == df_max_rodada['edicao.temporada'].max()].sort_values(by='pontos', ascending=False, inplace=False, ignore_index=True)

In [30]:
partidas_agendadas_df = partidas_agendadas_df.merge(df_max_rodada, on=['edicao.temporada', 'time'], how='left')

In [31]:
df_max_rodada.rename(columns={
    'time': 'adversario',
    'media_gols_marcados': 'media_gols_marcados_adversario',
    'media_gols_sofridos': 'media_gols_sofridos_adversario',
    'pontos_time': 'pontos_adversario',
    'aproveitamento': 'aproveitamento_adversario',
    'aproveitamento_recente': 'aproveitamento_recente_adversario',
    'confronto_recente': 'confronto_recente_adversario',
    }, inplace=True)
df_max_rodada.drop(columns=['pontos', 'jogos', 'pontos_acumulados', 'pontos_recentes', 'jogos_validos',], inplace=True)

In [32]:
partidas_agendadas_df = partidas_agendadas_df.merge(df_max_rodada, on=['edicao.temporada', 'adversario'], how='left')

In [33]:
partidas_model_df = partidas_model_df[partidas_model_df['status'] == 'finalizado']
partidas_model_df = pd.concat([partidas_model_df, partidas_agendadas_df], ignore_index=True)

In [34]:
colunas_descartar_partidas = ['rodada', 'estadio.nome_popular', 'edicao.edicao_id',
                              'ano-rodada', 'edicao.nome_popular', 'campeonato.campeonato_id',
                              'campeonato.nome', 'campeonato.slug',
                              'jogos', 'pontos_acumulados', "pontos_validos", "pontos_recentes", 
                              "jogos_validos", "pontos_shifted",
                              ]
partidas_model_final_df = partidas_model_df.drop(columns=[c for c in colunas_descartar_partidas if c in partidas_model_df.columns])

In [35]:
partidas_finalizadas_df = partidas_model_final_df[(partidas_model_final_df['status'] != 'agendado') &
                                                #   (partidas_model_final_df['mandante'] == 1) &
                                                  (partidas_model_final_df['edicao.temporada'] == partidas_model_final_df['edicao.temporada'].max())
                                                  ]

In [36]:
partidas_model_final_df = partidas_model_final_df[partidas_model_final_df["rodada_num"] != 1].reset_index(drop=True)


In [37]:
partidas_model_final_df[(partidas_model_final_df['status'] != 'finalizado') & (((partidas_model_final_df['time'] == 'Botafogo') & (partidas_model_final_df['edicao.temporada'] == 2025)))][
    [
        'edicao.temporada', 
        'rodada_num', 'time', 
        'adversario', 
        'mandante', 
        'pontos_time', 
        'confronto_recente', 
        'confronto_recente_adversario',
        'status',
    ]
]

edicao.temporada  rodada_num      time   adversario  mandante  \
2755              2025          31  Botafogo     Mirassol         0   
2770              2025          32  Botafogo        Vasco         1   
2807              2025          33  Botafogo      Vitória         0   
2816              2025          34  Botafogo        Sport         1   
2836              2025          35  Botafogo       Grêmio         1   
2859              2025          36  Botafogo  Corinthians         0   
2885              2025          37  Botafogo     Cruzeiro         0   
2896              2025          38  Botafogo    Fortaleza         1   

      pontos_time  confronto_recente  confronto_recente_adversario    status  
2755         47.0                2.2                           0.0  agendado  
2770         47.0                2.2                           2.0  agendado  
2807         47.0                2.2                           0.0  agendado  
2816         47.0                2.2                           3.0  agendado  
2836         47.0                2.2                           0.0  agendado  
2859         47.0                2.2                           1.0  agendado  
2885         47.0                2.2                           2.0  agendado  
2896         47.0                2.2                           0.4  agendado

### Modelo

In [38]:
# importando as bibliotecas
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBClassifier
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from sklearn.model_selection import TimeSeriesSplit
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

warnings.filterwarnings('ignore')

In [39]:
# filtra apenas partidas realizadas
df = partidas_model_final_df[(partidas_model_final_df["status"] != "agendado")].copy()

# remove colunas irrelevantes
exclude_cols = ["confronto_recente_adversario", "estadio.estadio_id", "data_realizacao", "status", "time", "adversario", "partida_id", "time_id", "adversario_id", "gols_time", "gols_adversario"]
df = df.drop(columns=exclude_cols)


com stratify=y e class_weight="balanced"

In [40]:
import time
partial_time = time.time()

# ================================
# 1️⃣ Pré-processamento
# ================================

# cria colunas de diferença (time - adversário)
df["diff_aproveitamento"] = df["aproveitamento"] - df["aproveitamento_adversario"]
df["diff_aproveitamento_recente"] = df["aproveitamento_recente"] - df["aproveitamento_recente_adversario"]
df["diff_media_gols"] = df["media_gols_marcados"] - df["media_gols_marcados_adversario"]
df["diff_media_gols_sofridos"] = df["media_gols_sofridos"] - df["media_gols_sofridos_adversario"]

# seleciona features
features = [
    "mandante",
    "rodada_num",
    "aproveitamento", "aproveitamento_recente",
    "aproveitamento_adversario", "aproveitamento_recente_adversario",
    "media_gols_marcados", "media_gols_sofridos",
    "media_gols_marcados_adversario", "media_gols_sofridos_adversario",
    "diff_aproveitamento", "diff_aproveitamento_recente",
    "diff_media_gols", "diff_media_gols_sofridos"
]

X = df[features].copy()
y = df["pontos"]  # variável alvo: 0,1,3

# converte mandante para numérico
X["mandante"] = X["mandante"].astype(int)

# ================================
# 2️⃣ Divide treino e teste
# ================================
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# ================================
# 3️⃣ Modelos de classificação
# ================================
models = {
    "RandomForest": RandomForestClassifier(n_estimators=200, random_state=42, class_weight="balanced"),
    "LogisticRegression": LogisticRegression(max_iter=1000, class_weight="balanced"),
    "GradientBoosting": GradientBoostingClassifier(n_estimators=200, random_state=42),
    "KNN": KNeighborsClassifier(n_neighbors=5),
    # "NaiveBayes": GaussianNB()
}

# ================================
# 4️⃣ Treino, previsão e métricas
# ================================
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    print(f"=== {name} ===")
    
    # Matriz de confusão (em tabela)
    cm = confusion_matrix(y_test, y_pred, labels=[0,1,3])
    cm_df = pd.DataFrame(cm, index=[0,1,3], columns=[0,1,3])
    print("Matriz de Confusão:")
    print(cm_df, "\n")
    
    # Classification report
    cr = classification_report(y_test, y_pred)
    print("Classification Report:")
    print(cr)
    print("="*50)

end_time = time.time()
execution_duration = end_time - partial_time
print(f"tempo de execução: {execution_duration} seconds")

=== RandomForest ===
Matriz de Confusão:
     0   1    3
0  107  26   69
1   55  20   72
3   59  31  111 

Classification Report:
              precision    recall  f1-score   support

         0.0       0.48      0.53      0.51       202
         1.0       0.26      0.14      0.18       147
         3.0       0.44      0.55      0.49       201

    accuracy                           0.43       550
   macro avg       0.39      0.41      0.39       550
weighted avg       0.41      0.43      0.41       550

=== LogisticRegression ===
Matriz de Confusão:
     0   1    3
0  111  37   54
1   58  17   72
3   55  24  122 

Classification Report:
              precision    recall  f1-score   support

         0.0       0.50      0.55      0.52       202
         1.0       0.22      0.12      0.15       147
         3.0       0.49      0.61      0.54       201

    accuracy                           0.45       550
   macro avg       0.40      0.42      0.41       550
weighted avg       0.42    

## Aplicação do modelo

### Leave-One-Season-Out (LOSO)
* O que é: Você considera cada temporada (ou grupo, ou indivíduo) como um “fold”.
* Como funciona: Em cada iteração, você usa todas as temporadas menos uma para treinar, e a temporada deixada de fora para validar.

In [41]:
partial_time = time.time()

# ================================
# 1️⃣ Features
# ================================
features = [
    "mandante", "rodada_num",
    "aproveitamento", "aproveitamento_recente",
    "aproveitamento_adversario", "aproveitamento_recente_adversario",
    "media_gols_marcados", "media_gols_sofridos",
    "media_gols_marcados_adversario", "media_gols_sofridos_adversario",
]

# ================================
# 2️⃣ Separar jogos realizados e agendados
# ================================
df_realizados = partidas_model_final_df[partidas_model_final_df["status"] != "agendado"].copy()
df_agendadas  = partidas_model_final_df[partidas_model_final_df["status"] == "agendado"].copy()

X_realizados = df_realizados[features].copy()
y_realizados = df_realizados["pontos"]
X_realizados["mandante"] = X_realizados["mandante"].astype(int)

X_agendadas = df_agendadas[features].copy()
X_agendadas["mandante"] = X_agendadas["mandante"].astype(int)

# ================================
# 3️⃣ Classificadores
# ================================
classifiers = {
    "RandomForest": RandomForestClassifier(n_estimators=200, class_weight="balanced", random_state=42),
    "LogisticRegression": LogisticRegression(max_iter=1000, class_weight="balanced", random_state=42),
    "GradientBoosting": GradientBoostingClassifier(n_estimators=200, random_state=42),
    "KNN": KNeighborsClassifier(n_neighbors=5),
    # "NaiveBayes": GaussianNB()
}

# ================================
# 4️⃣ Validação temporal Leave-One-Season-Out
# ================================
df_realizados = df_realizados.sort_values(["edicao.temporada", "rodada_num"])
temporadas = sorted(df_realizados["edicao.temporada"].unique())

print("=== Validação temporal Leave-One-Season-Out ===")
resultados = {}
for name, clf in classifiers.items():
    accs = []
    for i in range(1, len(temporadas)):
        train_seasons = temporadas[:i]   # até a temporada anterior
        test_season   = temporadas[i]    # temporada alvo

        X_train = df_realizados[df_realizados["edicao.temporada"].isin(train_seasons)][features].copy()
        y_train = df_realizados[df_realizados["edicao.temporada"].isin(train_seasons)]["pontos"]

        X_test  = df_realizados[df_realizados["edicao.temporada"] == test_season][features].copy()
        y_test  = df_realizados[df_realizados["edicao.temporada"] == test_season]["pontos"]

        X_train["mandante"] = X_train["mandante"].astype(int)
        X_test["mandante"]  = X_test["mandante"].astype(int)

        clf.fit(X_train, y_train)
        accs.append(clf.score(X_test, y_test))

    resultados[name] = {"acc_mean": np.mean(accs), "acc_std": np.std(accs)}
    print(f"{name}: média acc={np.mean(accs):.3f} ± {np.std(accs):.3f}")

# ================================
# 5️⃣ Seleciona o melhor modelo
# ================================
melhor_modelo = max(resultados, key=lambda k: resultados[k]["acc_mean"])
print(f"\n>>> Melhor modelo selecionado: {melhor_modelo} "
      f"(acc={resultados[melhor_modelo]['acc_mean']:.3f} ± {resultados[melhor_modelo]['acc_std']:.3f})")
melhor_modelo_loso_valor = resultados[melhor_modelo]['acc_mean']

end_time = time.time()
execution_duration = end_time - partial_time
print(f"tempo de execução: {execution_duration} seconds")

=== Validação temporal Leave-One-Season-Out ===
RandomForest: média acc=0.428 ± 0.051
LogisticRegression: média acc=0.467 ± 0.015
GradientBoosting: média acc=0.407 ± 0.055
KNN: média acc=0.410 ± 0.024

>>> Melhor modelo selecionado: LogisticRegression (acc=0.467 ± 0.015)
tempo de execução: 6.082149028778076 seconds


In [42]:
partial_time = time.time()

# ================================
# 6️⃣ Projeção futura (treina em todo histórico e prevê 2025+)
# ================================
loso_resumo_agendadas_df = df_agendadas[
    ["rodada_num", "partida_id", "mandante", "time", "adversario"]
].sort_values(
    by=["rodada_num", "partida_id", "mandante"], ascending=[True, True, False]
).reset_index(drop=True)

clf = classifiers[melhor_modelo]
clf.fit(X_realizados, y_realizados)  # treina no histórico inteiro
pred = clf.predict(X_agendadas)
loso_resumo_agendadas_df[f"{melhor_modelo}_pred"] = pred

print("\n=== Projeções futuras (2025+) ===")
print(loso_resumo_agendadas_df.head())

end_time = time.time()
execution_duration = end_time - partial_time
print(f"tempo de execução: {execution_duration} seconds")


=== Projeções futuras (2025+) ===
   rodada_num  partida_id  mandante        time  adversario  \
0          12       23453         1  Fluminense       Ceará   
1          12       23453         0       Ceará  Fluminense   
2          12       23462         1       Sport    Flamengo   
3          12       23462         0    Flamengo       Sport   
4          13       23466         1      Santos   Palmeiras   

   LogisticRegression_pred  
0                      3.0  
1                      0.0  
2                      3.0  
3                      0.0  
4                      3.0  
tempo de execução: 0.31423068046569824 seconds


### Monte Carlo Cross-Validation (MCCV)
* O que é: Também chamado de Repeated Random Sub-Sampling Validation.
* Como funciona: Você sorteia várias vezes divisões treino/teste (sempre respeitando a ordem temporal, no caso de séries temporais).

In [43]:
partial_time = time.time()

# ================================
# 1️⃣ Features
# ================================
features = [
    "mandante", "rodada_num",
    "aproveitamento", "aproveitamento_recente",
    "aproveitamento_adversario", "aproveitamento_recente_adversario",
    "media_gols_marcados", "media_gols_sofridos",
    "media_gols_marcados_adversario", "media_gols_sofridos_adversario",
]

# ================================
# 2️⃣ Separar jogos realizados e agendados
# ================================
df_realizados = partidas_model_final_df[partidas_model_final_df["status"] != "agendado"].copy()
X_realizados = df_realizados[features].copy()
y_realizados = df_realizados["pontos"]
X_realizados["mandante"] = X_realizados["mandante"].astype(int)

df_agendadas = partidas_model_final_df[partidas_model_final_df["status"] == "agendado"].copy()
X_agendadas = df_agendadas[features].copy()
X_agendadas["mandante"] = X_agendadas["mandante"].astype(int)

# ================================
# 3️⃣ Classificadores
# ================================
classifiers = {
    "RandomForest": RandomForestClassifier(n_estimators=200, class_weight="balanced"),
    "LogisticRegression": LogisticRegression(max_iter=1000, class_weight="balanced"),
    "GradientBoosting": GradientBoostingClassifier(n_estimators=200, random_state=42),
    "KNN": KNeighborsClassifier(n_neighbors=5),
    # "NaiveBayes": GaussianNB()
}

# ================================
# 4️⃣ Monte Carlo Validation
# ================================
n_splits = 1000
resultados = {name: [] for name in classifiers}

# Também vamos salvar previsões para jogos futuros em cada split
predicoes_futuras = {name: [] for name in classifiers}

print(f"=== Validação Monte Carlo ({n_splits} splits) ===")
for i in range(n_splits):
    X_train, X_test, y_train, y_test = train_test_split(
        X_realizados, y_realizados, test_size=0.2, stratify=y_realizados, random_state=i
    )
    for name, clf in classifiers.items():
        clf.fit(X_train, y_train)

        # avaliação
        y_pred = clf.predict(X_test)
        acc = accuracy_score(y_test, y_pred)
        resultados[name].append(acc)

        # projeções futuras
        pred = clf.predict(X_agendadas)
        predicoes_futuras[name].append(pred)

# calcular médias
for name in resultados:
    mean_acc = np.mean(resultados[name])
    std_acc = np.std(resultados[name])
    print(f"{name}: média acc={mean_acc:.3f} ± {std_acc:.3f}")
    resultados[name] = mean_acc

# escolher melhor modelo
best_model_name = max(resultados, key=resultados.get)
print(f"\n>>> Melhor modelo (Monte Carlo): {best_model_name} (acc_mean={resultados[best_model_name]:.3f})")
melhor_modelo_mccv_valor = resultados[best_model_name]

end_time = time.time()
execution_duration = end_time - partial_time
print(f"tempo de execução: {execution_duration} seconds")

=== Validação Monte Carlo (1000 splits) ===
RandomForest: média acc=0.442 ± 0.017
LogisticRegression: média acc=0.448 ± 0.019
GradientBoosting: média acc=0.436 ± 0.017
KNN: média acc=0.407 ± 0.018

>>> Melhor modelo (Monte Carlo): LogisticRegression (acc_mean=0.448)
tempo de execução: 3125.720350265503 seconds


In [44]:
partial_time = time.time()

# ================================
# 5️⃣ Projeção futura com frequências
# ================================
mccv_resumo_agendadas_df = df_agendadas[
    ["rodada_num", "partida_id", "mandante", "time", "adversario"]
].sort_values(by=["rodada_num", "partida_id", "mandante"], ascending=[True, True, False]).reset_index(drop=True)

# Pega todas as previsões do melhor modelo
all_preds = np.array(predicoes_futuras[best_model_name])  # n_splits x n_partidas
all_preds = all_preds.T  # n_partidas x n_splits

# resultado mais frequente (modo)
modo = [np.bincount(row.astype(int)).argmax() for row in all_preds]

# distribuição percentual
freq_percent = [
    {p: (np.sum(row == p) / n_splits) * 100 for p in [0, 1, 3]} 
    for row in all_preds
]

# adiciona ao dataframe final
mccv_resumo_agendadas_df[f"{best_model_name}_pred"] = modo
mccv_resumo_agendadas_df[f"{best_model_name}_freq"] = freq_percent

print("\n=== Projeções futuras com o melhor modelo (Monte Carlo) ===")
print(mccv_resumo_agendadas_df.head())

end_time = time.time()
execution_duration = end_time - partial_time
print(f"tempo de execução: {execution_duration} seconds")


=== Projeções futuras com o melhor modelo (Monte Carlo) ===
   rodada_num  partida_id  mandante        time  adversario  \
0          12       23453         1  Fluminense       Ceará   
1          12       23453         0       Ceará  Fluminense   
2          12       23462         1       Sport    Flamengo   
3          12       23462         0    Flamengo       Sport   
4          13       23466         1      Santos   Palmeiras   

   LogisticRegression_pred     LogisticRegression_freq  
0                        3  {0: 0.0, 1: 0.0, 3: 100.0}  
1                        0   {0: 99.9, 1: 0.1, 3: 0.0}  
2                        3  {0: 0.0, 1: 0.0, 3: 100.0}  
3                        0  {0: 100.0, 1: 0.0, 3: 0.0}  
4                        3  {0: 0.0, 1: 0.0, 3: 100.0}  
tempo de execução: 0.01027822494506836 seconds


In [45]:
modelos = [best_model_name + "_pred"]

for modelo in modelos:
    mccv_resumo_agendadas_df[f"{modelo}_freq"] = mccv_resumo_agendadas_df.apply(
        lambda row: row[modelo.replace("_pred", "_freq")].get(row[modelo], 0),
        axis=1
    )


In [46]:
def gerar_grafico_percentual(freq_dict):
    # freq_dict = {0: xx%, 1: xx%, 3: xx%}
    total_blocos = 10  # define o tamanho do gráfico
    grafico = ""
    for p in [0,1,3]:
        n_blocos = int((freq_dict.get(p,0)/100)*total_blocos)
        grafico += f"{p}:{'█'*n_blocos}{' '*(total_blocos-n_blocos)} "
    return grafico.strip()

# Aplicando no DataFrame resumo_agendadas
for name in [best_model_name]:
    mccv_resumo_agendadas_df[f"{name}_grafico"] = mccv_resumo_agendadas_df[f"{name}_freq"].apply(gerar_grafico_percentual)

# Exibe tabela final compacta
cols = []
for name in [best_model_name]:
    cols += [f"{name}_pred", f"{name}_grafico"]

# mccv_resumo_agendadas_df[cols]



### Adicionando previsões aos resultados

##### LOSO

In [47]:
# modelos = ['RandomForest_pred', 'LogisticRegression_pred']
modelos = [melhor_modelo + "_pred"]

tabela_evolucao_df = None
for modelo in modelos:
    temp_df = pd.concat(
        [
            partidas_finalizadas_df[['rodada_num', 'time', 'pontos']], 
            loso_resumo_agendadas_df[['rodada_num', 'time', modelo]].rename(columns={modelo: 'pontos'})
        ], ignore_index=True)
    temp_df = temp_df.sort_values(by=['time', 'rodada_num'], ascending=[True, True]).reset_index(drop=True)
    temp_df['pontos'] = temp_df.groupby(['time'])["pontos"].cumsum()
    temp_df['modelo'] = modelo.replace('_pred', '_LOSO')
    tabela_evolucao_df = pd.concat([tabela_evolucao_df, temp_df], ignore_index=True)

# calcula posição por rodada e modelo
tabela_evolucao_df.sort_values(by=['rodada_num', 'modelo', 'pontos'], ascending=[True, True, False], inplace=True)
tabela_evolucao_df["posicao"] = (
    tabela_evolucao_df
    .groupby(["rodada_num", "modelo"])["pontos"]
    .rank(method="min", ascending=False)  # menor rank = melhor posição
    .astype(int)
)

#### MCCV

In [48]:
# modelos = ['RandomForest_pred', 'LogisticRegression_pred']
modelos = [best_model_name + "_pred"]

# tabela_evolucao_df = None
for modelo in modelos:
    temp_df = pd.concat(
        [
            partidas_finalizadas_df[['rodada_num', 'time', 'pontos']], 
            mccv_resumo_agendadas_df[['rodada_num', 'time', modelo]].rename(columns={modelo: 'pontos'})
        ], ignore_index=True)
    temp_df = temp_df.sort_values(by=['time', 'rodada_num'], ascending=[True, True]).reset_index(drop=True)
    temp_df['pontos'] = temp_df.groupby(['time'])["pontos"].cumsum()
    temp_df['modelo'] = modelo.replace('_pred', '_MCCV')
    tabela_evolucao_df = pd.concat([tabela_evolucao_df, temp_df], ignore_index=True)

# calcula posição por rodada e modelo
tabela_evolucao_df.sort_values(by=['rodada_num', 'modelo', 'pontos'], ascending=[True, True, False], inplace=True)
tabela_evolucao_df["posicao"] = (
    tabela_evolucao_df
    .groupby(["rodada_num", "modelo"])["pontos"]
    .rank(method="min", ascending=False)  # menor rank = melhor posição
    .astype(int)
)

In [49]:
tabela_evolucao_df[(tabela_evolucao_df['rodada_num'] == 38)]
# tabela_evolucao_df[(tabela_evolucao_df['rodada_num'] == 23) & (tabela_evolucao_df['time'] == 'Botafogo')]

rodada_num           time  pontos                   modelo  posicao
740           38       Flamengo    76.0  LogisticRegression_LOSO        1
741           38      Palmeiras    74.0  LogisticRegression_LOSO        2
742           38       Cruzeiro    72.0  LogisticRegression_LOSO        3
743           38       Mirassol    67.0  LogisticRegression_LOSO        4
744           38       Botafogo    62.0  LogisticRegression_LOSO        5
745           38     Fluminense    62.0  LogisticRegression_LOSO        5
746           38          Bahia    61.0  LogisticRegression_LOSO        7
747           38      São Paulo    56.0  LogisticRegression_LOSO        8
748           38    Atlético-MG    54.0  LogisticRegression_LOSO        9
749           38          Vasco    54.0  LogisticRegression_LOSO        9
750           38         Grêmio    49.0  LogisticRegression_LOSO       11
751           38    Corinthians    48.0  LogisticRegression_LOSO       12
752           38  Internacional    47.0  LogisticRegression_LOSO       13
753           38         Santos    47.0  LogisticRegression_LOSO       13
754           38        Vitória    43.0  LogisticRegression_LOSO       15
755           38     Bragantino    42.0  LogisticRegression_LOSO       16
756           38          Ceará    41.0  LogisticRegression_LOSO       17
757           38      Juventude    41.0  LogisticRegression_LOSO       17
758           38      Fortaleza    37.0  LogisticRegression_LOSO       19
759           38          Sport    29.0  LogisticRegression_LOSO       20
1063          38       Flamengo    76.0  LogisticRegression_MCCV        1
1329          38      Palmeiras    74.0  LogisticRegression_MCCV        2
1025          38       Cruzeiro    72.0  LogisticRegression_MCCV        3
1291          38       Mirassol    67.0  LogisticRegression_MCCV        4
873           38       Botafogo    62.0  LogisticRegression_MCCV        5
1101          38     Fluminense    62.0  LogisticRegression_MCCV        5
835           38          Bahia    61.0  LogisticRegression_MCCV        7
1443          38      São Paulo    56.0  LogisticRegression_MCCV        8
797           38    Atlético-MG    54.0  LogisticRegression_MCCV        9
1481          38          Vasco    54.0  LogisticRegression_MCCV        9
1177          38         Grêmio    49.0  LogisticRegression_MCCV       11
987           38    Corinthians    48.0  LogisticRegression_MCCV       12
1215          38  Internacional    47.0  LogisticRegression_MCCV       13
1367          38         Santos    47.0  LogisticRegression_MCCV       13
1519          38        Vitória    43.0  LogisticRegression_MCCV       15
911           38     Bragantino    42.0  LogisticRegression_MCCV       16
949           38          Ceará    41.0  LogisticRegression_MCCV       17
1253          38      Juventude    41.0  LogisticRegression_MCCV       17
1139          38      Fortaleza    37.0  LogisticRegression_MCCV       19
1405          38          Sport    29.0  LogisticRegression_MCCV       20

In [50]:

mccv_resumo_agendadas_df[['rodada_num', 'mandante', 'time', 'adversario', f'{best_model_name}_pred', f'{best_model_name}_pred_freq']]

# mccv_resumo_agendadas_df.columns

rodada_num  mandante        time  adversario  LogisticRegression_pred  \
0            12         1  Fluminense       Ceará                        3   
1            12         0       Ceará  Fluminense                        0   
2            12         1       Sport    Flamengo                        3   
3            12         0    Flamengo       Sport                        0   
4            13         1      Santos   Palmeiras                        3   
..          ...       ...         ...         ...                      ...   
163          38         0   São Paulo     Vitória                        0   
164          38         1       Ceará   Palmeiras                        3   
165          38         0   Palmeiras       Ceará                        0   
166          38         1       Sport      Grêmio                        3   
167          38         0      Grêmio       Sport                        0   

     LogisticRegression_pred_freq  
0                           100.0  
1                            99.9  
2                           100.0  
3                           100.0  
4                           100.0  
..                            ...  
163                          55.0  
164                          99.2  
165                          99.9  
166                         100.0  
167                         100.0  

[168 rows x 6 columns]

In [51]:
loso_resumo_agendadas_df[['rodada_num', 'mandante', 'time', 'adversario', f'{melhor_modelo}_pred']]

# mccv_resumo_agendadas_df.columns

rodada_num  mandante        time  adversario  LogisticRegression_pred
0            12         1  Fluminense       Ceará                      3.0
1            12         0       Ceará  Fluminense                      0.0
2            12         1       Sport    Flamengo                      3.0
3            12         0    Flamengo       Sport                      0.0
4            13         1      Santos   Palmeiras                      3.0
..          ...       ...         ...         ...                      ...
163          38         0   São Paulo     Vitória                      0.0
164          38         1       Ceará   Palmeiras                      3.0
165          38         0   Palmeiras       Ceará                      0.0
166          38         1       Sport      Grêmio                      3.0
167          38         0      Grêmio       Sport                      0.0

[168 rows x 5 columns]

In [52]:
# unindo resultados finais e previsões
partidas_finalizadas_df = partidas_finalizadas_df[['rodada_num', 'mandante', 'time', 'adversario', 'pontos']]
partidas_finalizadas_df['status'] = 'Finalizado'

loso_resumo_agendadas_df['validacao'] = 'LOSO'
mccv_resumo_agendadas_df['validacao'] = 'MCCV'

# modelos = [name + "_pred" for name in classifiers.keys()]
# modelos_frequencia = [x + "_freq" for x in modelos]

modelos_loso = [melhor_modelo + "_pred"]
modelos_mccv = [best_model_name + "_pred"]
modelos_frequencia = [x + "_freq" for x in modelos_mccv]

colunas_loso = ['rodada_num', 'mandante', 'time', 'adversario', 'validacao'] + modelos_loso
colunas_mccv = ['rodada_num', 'mandante', 'time', 'adversario', 'validacao'] + modelos_mccv + modelos_frequencia

partidas_predicao_df = pd.concat(
    [
        loso_resumo_agendadas_df[colunas_loso],
        mccv_resumo_agendadas_df[colunas_mccv]
    ], ignore_index=True)
partidas_predicao_df['status'] = 'Previsto'


partidas_df = None

# modelos = modelos_mccv + modelos_loso
# for modelo in modelos:
#     if modelo in modelos_mccv:
#         temp = partidas_predicao_df[['rodada_num', 'mandante', 'time', 'adversario', modelo, f'{modelo}_freq', 'status', 'validacao']].copy()
#         temp = temp[temp['validacao'] == 'MCCV']
#         temp.rename(columns={modelo: 'pontos', f'{modelo}_freq': 'frequencia'}, inplace=True)
#         temp['modelo'] = temp.apply(lambda x: f'{modelo}'.replace('_pred', '') + ('_MCCV' if x['validacao'] == 'MCCV' else '_LOSO'), axis=1)
#         partidas_df = pd.concat([partidas_df, temp], ignore_index=True)
#     else:
#         temp = partidas_predicao_df[['rodada_num', 'mandante', 'time', 'adversario', modelo, 'status', 'validacao']].copy()
#         temp = temp[temp['validacao'] == 'LOSO']
#         temp.rename(columns={modelo: 'pontos'}, inplace=True)
#         temp['modelo'] = temp.apply(lambda x: f'{modelo}'.replace('_pred', '') + ('_MCCV' if x['validacao'] == 'MCCV' else '_LOSO'), axis=1)
#         partidas_df = pd.concat([partidas_df, temp], ignore_index=True)

# Loop para MCCV
for modelo in modelos_mccv:
    temp = partidas_predicao_df[['rodada_num', 'mandante', 'time', 'adversario', modelo, f'{modelo}_freq', 'status', 'validacao']].copy()
    temp = temp[temp['validacao'] == 'MCCV']
    temp.rename(columns={modelo: 'pontos', f'{modelo}_freq': 'frequencia'}, inplace=True)
    temp['modelo'] = f'{modelo.replace("_pred", "")}_MCCV'
    partidas_df = pd.concat([partidas_df, temp], ignore_index=True)

# Loop para LOSO
for modelo in modelos_loso:
    temp = partidas_predicao_df[['rodada_num', 'mandante', 'time', 'adversario', modelo, 'status', 'validacao']].copy()
    temp = temp[temp['validacao'] == 'LOSO']
    temp.rename(columns={modelo: 'pontos'}, inplace=True)
    temp['frequencia'] = None
    temp['modelo'] = f'{modelo.replace("_pred", "")}_LOSO'
    partidas_df = pd.concat([partidas_df, temp], ignore_index=True)

partidas_df = pd.concat([partidas_df, partidas_finalizadas_df], ignore_index=True)
partidas_df['resultado'] = partidas_df.apply(lambda x: 'Vitória' if x['pontos'] == 3 else ('Empate' if x['pontos'] == 1 else 'Derrota'), axis=1)
# partidas_df['resultado'] = partidas_df['pontos']
partidas_df['mandante'] = partidas_df['mandante'].apply(lambda x: 'Sim' if x == 1 else 'Não')
partidas_df.sort_values(by=['rodada_num', 'time', 'mandante'], ascending=[True, True, False], inplace=True)
partidas_df = partidas_df[['rodada_num', 'status', 'mandante', 'time', 'adversario', 'resultado', 'frequencia', 'modelo']]

In [53]:
if melhor_modelo_loso_valor > melhor_modelo_mccv_valor:
    filtro_modelo = modelos_loso[0].replace("_pred", "") + "_LOSO"
else:
    filtro_modelo = modelos_mccv[0].replace("_pred", "") + "_MCCV"

print(f"O melhor modelo é: {filtro_modelo}")

O melhor modelo é: LogisticRegression_LOSO


In [54]:
# partidas_df = partidas_df[(partidas_df['modelo'] == filtro_modelo) | (partidas_df['status'] != 'Previsto')]
partidas_df = partidas_df[(partidas_df['modelo'] == filtro_modelo)]
tabela_evolucao_df = tabela_evolucao_df[(tabela_evolucao_df['modelo'] == filtro_modelo)]

In [55]:
# add coluna com data de atualização
tabela_evolucao_df['data_atualizacao'] = pd.Timestamp.now()

In [56]:
# estatisticas
times_stats = partidas_model_final_df[(partidas_model_final_df['status']=='agendado')]
colunas = ['time', 'pontos_time', 'aproveitamento', 'aproveitamento_recente', 'media_gols_marcados', 'media_gols_sofridos']
times_stats = times_stats[colunas].drop_duplicates()

# add qtd_jogos
qtd_jogos = partidas_df[partidas_df['status'] != 'previsto'].groupby('time').count().reset_index().rename(columns={'status': 'partidas_previstas'})
qtd_jogos['jogos'] = 38 - qtd_jogos['partidas_previstas']
qtd_jogos = qtd_jogos[['time', 'jogos']]
times_stats = times_stats.merge(qtd_jogos, on='time', how='left')
times_stats['jogos'] = times_stats['jogos'].fillna(0).astype(int)

In [57]:
tabela_evolucao_df.to_csv("app/tabela-evolucao-modelos.csv", index=False)
partidas_df.to_csv("app/partidas-modelos.csv", index=False)
times_stats.to_csv("app/times-stats.csv", index=False)


In [58]:
end_time = time.time()
execution_duration = end_time - start_time
print(f"tempo de execução total: {execution_duration} segundos")

tempo de execução total: 3143.2680366039276 segundos
